# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from datetime import datetime

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_df = pd.read_csv("../output_data/cities(out).csv")
output_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temperature,Latitude,Longitude,Wind Speed
0,0,Punta Arenas,0,CL,1612552202,38,59.00,-53.1500,-70.9167,28.77
1,1,Naryan-Mar,57,RU,1612552438,89,-12.53,67.6713,53.0870,11.54
2,2,Coquimbo,20,CL,1612552375,72,66.20,-29.9533,-71.3436,13.80
3,3,Faanui,9,PF,1612552440,66,81.18,-16.4833,-151.7500,6.24
4,4,New Norfolk,100,AU,1612552441,92,64.99,-42.7826,147.0587,3.44
...,...,...,...,...,...,...,...,...,...,...
583,583,Örnsköldsvik,0,SE,1612552931,86,15.80,63.2909,18.7153,6.91
584,584,Chimbote,4,PE,1612552953,59,76.46,-9.0853,-78.5783,11.27
585,585,Lodja,100,CD,1612553105,70,77.45,-3.4833,23.4333,0.47
586,586,Tavira,40,PT,1612553106,87,55.00,37.1273,-7.6486,11.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = output_df[["Latitude", "Longitude"]].astype(float)
weight = output_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure(center=(30.0, -5.0), zoom_level=2)
max_intensity = np.max(weight)
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity= max_intensity,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#find cities with ideal weather
#max temp lower than 80, higher than 70
#Wind less than 10 mph
#Zero cloudiness
ideal_df = output_df[(output_df['Temperature']<=80) & (output_df['Temperature']>=70) & (output_df['Wind Speed']<=10) & (output_df['Cloudiness']==0)] 


#Drop rows that don't contain all three conditions 


del ideal_df['Unnamed: 0']

ideal_df.reset_index(drop=True)
ideal_df.set_index("City")
converted_time = pd.to_datetime(ideal_df['Date'], unit='s')
del ideal_df['Date']

#replace unix time with converted time 

ideal_df['Date/Time'] = converted_time

ideal_df.reset_index(drop=True)

<ipython-input-6-479488763fb6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df['Date/Time'] = converted_time


,City,Cloudiness,Country,Humidity,Temperature,Latitude,Longitude,Wind Speed,Date/Time
0,Poum,0,NC,76,79.27,-20.2333,164.0167,6.67,2021-02-05 19:14:39
1,Castro,0,BR,53,75.56,-24.7911,-50.0119,6.42,2021-02-05 19:12:55
2,North Port,0,US,52,73.99,27.0442,-82.2359,5.99,2021-02-05 19:14:53
3,Riyadh,0,SA,17,71.60,24.6877,46.7219,9.22,2021-02-05 19:15:22
4,Mar del Plata,0,AR,46,75.20,-38.0023,-57.5575,6.91,2021-02-05 19:15:00
5,Cap Malheureux,0,MU,84,75.99,-19.9842,57.6142,1.01,2021-02-05 19:16:43
6,Bhatkal,0,IN,68,75.56,13.9667,74.5667,5.61,2021-02-05 19:14:44
7,Kousséri,0,CM,12,78.80,12.0769,15.0306,6.91,2021-02-05 19:18:53
8,Gouré,0,NE,13,75.99,13.9835,10.2704,5.99,2021-02-05 19:19:29
9,Dubai,0,AE,46,73.00,25.2582,55.3047,6.91,2021-02-05 19:20:42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_df.loc[:,["City","Country", "Latitude", "Longitude"]]

hotel_df["Hotel Name"] = ""

hotel_df.reset_index(drop=True)


,City,Country,Latitude,Longitude,Hotel Name
0,Poum,NC,-20.2333,164.0167,
1,Castro,BR,-24.7911,-50.0119,
2,North Port,US,27.0442,-82.2359,
3,Riyadh,SA,24.6877,46.7219,
4,Mar del Plata,AR,-38.0023,-57.5575,
5,Cap Malheureux,MU,-19.9842,57.6142,
6,Bhatkal,IN,13.9667,74.5667,
7,Kousséri,CM,12.0769,15.0306,
8,Gouré,NE,13.9835,10.2704,
9,Dubai,AE,25.2582,55.3047,


In [8]:
# find hotels

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "lodging",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat/long from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]

    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
     #if there is no hotel available, print message
    except (KeyError, IndexError):
        print("No hotel nearby... skipping.")
        
    print("------------")

Retrieving Results for Index 38: Poum.
No hotel nearby... skipping.
------------
Retrieving Results for Index 63: Castro.
Closest hotel in Castro is Pousada Rota Dos Tropeiros.
------------
Retrieving Results for Index 70: North Port.
Closest hotel in North Port is Hampton Inn & Suites North Port.
------------
Retrieving Results for Index 81: Riyadh.
Closest hotel in Riyadh is Four Seasons Hotel Riyadh At Kingdom Center.
------------
Retrieving Results for Index 137: Mar del Plata.
Closest hotel in Mar del Plata is Hotel Las Rocas.
------------
Retrieving Results for Index 145: Cap Malheureux.
Closest hotel in Cap Malheureux is LUX Grand Gaube Resort & Villas.
------------
Retrieving Results for Index 190: Bhatkal.
Closest hotel in Bhatkal is The Royal Oak Hotel.
------------
Retrieving Results for Index 260: Kousséri.
Closest hotel in Kousséri is Radisson Blu Hotel, N'Djamena.
------------
Retrieving Results for Index 292: Gouré.
No hotel nearby... skipping.
------------
Retrieving Re

In [10]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
38,Poum,NC,-20.2333,164.0167,
63,Castro,BR,-24.7911,-50.0119,Pousada Rota Dos Tropeiros
70,North Port,US,27.0442,-82.2359,Hampton Inn & Suites North Port
81,Riyadh,SA,24.6877,46.7219,Four Seasons Hotel Riyadh At Kingdom Center
137,Mar del Plata,AR,-38.0023,-57.5575,Hotel Las Rocas
145,Cap Malheureux,MU,-19.9842,57.6142,LUX Grand Gaube Resort & Villas
190,Bhatkal,IN,13.9667,74.5667,The Royal Oak Hotel
260,Kousséri,CM,12.0769,15.0306,"Radisson Blu Hotel, N'Djamena"
292,Gouré,NE,13.9835,10.2704,
423,Dubai,AE,25.2582,55.3047,Rove La Mer Beach


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(hotel_markers)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))